In [17]:
%pip install nbformat
%pip install imbalanced-learn

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [18]:
import nbformat

# Read your notebook (assuming version 4 for example purposes)
nb = nbformat.read("ml_project.ipynb", as_version=4)

# Normalize the notebook to add missing id fields and other updates
nbformat.validator.validate(nb)

# Write the normalized notebook back to a file
nbformat.write(nb, "ml_project_normalized.ipynb")

In [20]:
%run ml_project_normalized.ipynb

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
dropped columns
cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
dropped columns


/var/folders/yf/n0zzld6945vfyf6hj9_n1g4c0000gn/T/ipykernel_45769/850107665.py:4: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  dt_series = pd.to_datetime(df['intake_time'], errors='coerce')


cleaned intake time
cleaned intake condition
cleaned age and sex
cleaned breed
cleaned color
Done running ml_project.ipynb.


In [21]:
from sklearn.preprocessing import OneHotEncoder, FunctionTransformer
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from sklearn.model_selection import RandomizedSearchCV, cross_val_score
from xgboost import XGBClassifier

def train_classifier(X_train, y_train, X_test, rare_classes):
    """
    Trains an XGBoost model using a pipeline that includes frequency encoding,
    OneHotEncoder, SMOTE for class balancing, and hyperparameter tuning.
    
    Parameters:
        X_train (pd.DataFrame): Training features.
        y_train (pd.Series or np.array): Training target values.
        X_test (pd.DataFrame): Test features.
        rare_classes (list): List of integer-encoded classes to be oversampled with SMOTE.
    
    Returns:
        best_estimator: The best estimator from RandomizedSearchCV.
        test_predictions: The predicted labels for X_test from the best estimator.
    """
    # Define SMOTE only for rare classes
    smote = SMOTE(sampling_strategy={cls: int(sum(y_train == cls) * 3) for cls in rare_classes}, random_state=42)

    pipeline = Pipeline([
        ('freq', FunctionTransformer(apply_freq_encode, validate=False)),
        ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False)),
        ('smote', smote),
        ('xgb', XGBClassifier(eval_metric='logloss', verbosity=1))
    ])
    
    param_distributions = {
        "xgb__max_depth": [3, 6, 9],
        "xgb__learning_rate": [0.01, 0.1, 0.2],
        "xgb__n_estimators": [50, 100, 200],
        "xgb__subsample": [0.5, 0.7, 1.0],
        "xgb__colsample_bytree": [0.5, 0.7, 1.0]
    }
    
    randomized_search = RandomizedSearchCV(
        estimator=pipeline,
        param_distributions=param_distributions,
        n_iter=1,
        cv=5,
        scoring='accuracy',
        verbose=3
    )
    
    randomized_search.fit(X_train, y_train)
    
    print('Best parameters:', randomized_search.best_params_)
    print('Best cross-validation accuracy:', randomized_search.best_score_)
    
    cv_scores = cross_val_score(randomized_search.best_estimator_, X_train, y_train, cv=5, verbose=3, scoring=make_scorer(classification_report_with_accuracy_score))
    print('Generalization accuracy (via cross_val_score):', cv_scores.mean())
    
    test_predictions = randomized_search.predict(X_test)
    
    return randomized_search.best_estimator_, test_predictions


In [22]:
# from sklearn.utils import resample
# df_train_downsample = resample(df_train, replace=True, n_samples=10000, random_state=42)
# print(df_train_downsample.shape)

In [23]:
from sklearn.preprocessing import LabelEncoder
from sklearn.utils import resample

df_train = resample(df_train, replace=True, n_samples=25000, random_state=42)

## Encode targets with LabelEncoder
# Dog encoding
le = LabelEncoder()
X_train = df_train.drop(columns=['outcome_type'])

y_train = df_train['outcome_type']
y_train = le.fit_transform(y_train)
print('Encoding mapping:', le.classes_)
rare_classes = [le.transform(['Euthanasia', 'Died'])[i] for i in range(2) if 'Euthanasia' in le.classes_ and 'Died' in le.classes_]

print("Training model for Dog data:")
best_estimator, predictions_encoded = train_classifier(X_train, y_train, df_test, rare_classes)
predictions = le.inverse_transform(predictions_encoded)

save_predictions(predictions)

Encoding mapping: ['Adoption' 'Died' 'Euthanasia' 'Return to Owner' 'Transfer']
Training model for Dog data:
Fitting 5 folds for each of 1 candidates, totalling 5 fits


/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


[CV 1/5] END xgb__colsample_bytree=0.7, xgb__learning_rate=0.2, xgb__max_depth=3, xgb__n_estimators=50, xgb__subsample=0.5;, score=0.606 total time=  16.7s


/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


[CV 2/5] END xgb__colsample_bytree=0.7, xgb__learning_rate=0.2, xgb__max_depth=3, xgb__n_estimators=50, xgb__subsample=0.5;, score=0.611 total time=  17.2s


...

              precision    recall  f1-score   support

           0       0.61      0.85      0.71      2464
           1       0.00      0.00      0.00        47
           2       0.34      0.20      0.25       152
           3       0.58      0.53      0.55       755
           4       0.68      0.32      0.44      1582

    accuracy                           0.61      5000
   macro avg       0.44      0.38      0.39      5000
weighted avg       0.61      0.61      0.58      5000

[CV] END ................................ score: (test=0.381) total time=  15.4s


/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

              precision    recall  f1-score   support

           0       0.60      0.84      0.70      2464
           1       0.00      0.00      0.00        47
           2       0.28      0.24      0.26       152
           3       0.58      0.54      0.56       755
           4       0.63      0.29      0.40      1582

    accuracy                           0.60      5000
   macro avg       0.42      0.38      0.38      5000
weighted avg       0.59      0.60      0.56      5000

[CV] END ................................ score: (test=0.381) total time=  16.4s


/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/caseyc/Library/Python/3.9/lib/python/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(resu

              precision    recall  f1-score   support

           0       0.59      0.85      0.70      2464
           1       0.00      0.00      0.00        47
           2       0.27      0.24      0.25       152
           3       0.60      0.50      0.55       755
           4       0.66      0.29      0.40      1582

    accuracy                           0.59      5000
   macro avg       0.42      0.38      0.38      5000
weighted avg       0.60      0.59      0.56      5000

[CV] END ................................ score: (test=0.376) total time=  17.1s
Generalization accuracy (via cross_val_score): 0.38311878097775587
Combined test predictions saved to: ./test_predictions_combined.csv
